Classification of the bacteria data using three different classifiers:
1. SVM
2. RandomForest with decision stumps
3. AdaBoost with decision stumps.

The data is: each permutation of species, replicate and growth phase along the rows; and the fluorescense spectra readings and growth phase along the columns. 

In [1]:
import os

import numpy as np
import pandas as pd

In [2]:
os.getcwd()

'/mnt/d/dev/summer-bursary-2018/bacteria'

In [3]:
os.listdir('data')

['16ms_32ms_growth_phase_spectra.csv',
 '16_ms_lag_codes.csv',
 'bacteria.csv',
 'Classific.py',
 'graphs.pdf',
 'wavelengths.csv']

In [4]:
df = pd.read_csv('data/bacteria.csv', header=[0, 1, 2], index_col=0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1043 entries, 0 to 1042
Columns: 135 entries, (lag, bc, 01) to (stat, se, 07)
dtypes: float64(135)
memory usage: 1.1 MB


# Preprocessing

In [5]:
X = df.reorder_levels([1, 0, 2], axis=1).sort_index(axis=1).T
X = (X - X.values.min()) / (X.values.max() - X.values.min())

print(X.values.min(), X.values.mean(), X.values.max())
X.head()

0.0 0.042780876819567694 1.0


0         1         2         3     \
species growth_phase replicate                                           
bc      lag          01         0.001977  0.002037  0.002005  0.001957   
                     02         0.002001  0.001896  0.001979  0.002061   
                     03         0.001980  0.002002  0.001937  0.001898   
                     04         0.001935  0.001879  0.001942  0.002010   
                     05         0.001920  0.001956  0.002090  0.002094   

                                    4         5         6         7     \
species growth_phase replicate                                           
bc      lag          01         0.001982  0.001897  0.001977  0.002017   
                     02         0.002049  0.002020  0.002011  0.001924   
                     03         0.001922  0.002017  0.001900  0.001942   
                     04         0.002120  0.002036  0.002010  0.001897   
                     05         0.002092  0.002092  0.001977  0.001939   

                                    8         9       ...         1033  \
species growth_phase replicate                        ...                
bc      lag          01         0.002025  0.001970    ...     0.002022   
                     02         0.001903  0.002003    ...     0.001963   
                     03         0.002091  0.002017    ...     0.002096   
                     04         0.001916  0.001989    ...     0.002141   
                     05         0.001941  0.001900    ...     0.002150   

                                    1034      1035      1036      1037  \
species growth_phase replicate                                           
bc      lag          01         0.002082  0.002125  0.002045  0.002025   
                     02         0.001970  0.001888  0.001982  0.002008   
                     03         0.002161  0.002161  0.002141  0.002059   
                     04         0.002104  0.002010  0.002073  0.002111   
                     05         0.002073  0.002114  0.002018  0.002130   

                                    1038      1039      1040      1041  \
species growth_phase replicate                                           
bc      lag          01         0.002145  0.001997  0.002005  0.001965   
                     02         0.002018  0.002080  0.002001  0.001999   
                     03         0.001980  0.001880  0.001997  0.001942   
                     04         0.002204  0.002036  0.002057  0.001982   
                     05         0.002226  0.001941  0.001979  0.001960   

                                    1042  
species growth_phase replicate            
bc      lag          01         0.001957  
                     02         0.001960  
                     03         0.002019  
                     04         0.002019  
                     05         0.002032  

[5 rows x 1043 columns]

In [6]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

growth_phases = X.reset_index()['growth_phase']
growth_phases = growth_phases.values  # get the numpy array
growth_phases = growth_phases.reshape(-1, 1)

print(growth_phases[:5])

le = LabelEncoder()
growth_phases = le.fit_transform(growth_phases)

ohe = OneHotEncoder(sparse=False)
growth_phases = ohe.fit_transform(growth_phases.reshape(-1, 1))

print(growth_phases[:5])

[['lag']
 ['lag']
 ['lag']
 ['lag']
 ['lag']]
[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]


/home/anthony/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [7]:
X['lag'] = growth_phases[:, 0]
X['log'] = growth_phases[:, 1]
X['stat'] = growth_phases[:, 2]

X.head()

0         1         2         3  \
species growth_phase replicate                                           
bc      lag          01         0.001977  0.002037  0.002005  0.001957   
                     02         0.002001  0.001896  0.001979  0.002061   
                     03         0.001980  0.002002  0.001937  0.001898   
                     04         0.001935  0.001879  0.001942  0.002010   
                     05         0.001920  0.001956  0.002090  0.002094   

                                       4         5         6         7  \
species growth_phase replicate                                           
bc      lag          01         0.001982  0.001897  0.001977  0.002017   
                     02         0.002049  0.002020  0.002011  0.001924   
                     03         0.001922  0.002017  0.001900  0.001942   
                     04         0.002120  0.002036  0.002010  0.001897   
                     05         0.002092  0.002092  0.001977  0.001939   

                                       8         9  ...       1036      1037  \
species growth_phase replicate                      ...                        
bc      lag          01         0.002025  0.001970  ...   0.002045  0.002025   
                     02         0.001903  0.002003  ...   0.001982  0.002008   
                     03         0.002091  0.002017  ...   0.002141  0.002059   
                     04         0.001916  0.001989  ...   0.002073  0.002111   
                     05         0.001941  0.001900  ...   0.002018  0.002130   

                                    1038      1039      1040      1041  \
species growth_phase replicate                                           
bc      lag          01         0.002145  0.001997  0.002005  0.001965   
                     02         0.002018  0.002080  0.002001  0.001999   
                     03         0.001980  0.001880  0.001997  0.001942   
                     04         0.002204  0.002036  0.002057  0.001982   
                     05         0.002226  0.001941  0.001979  0.001960   

                                    1042  lag  log  stat  
species growth_phase replicate                            
bc      lag          01         0.001957  1.0  0.0   0.0  
                     02         0.001960  1.0  0.0   0.0  
                     03         0.002019  1.0  0.0   0.0  
                     04         0.002019  1.0  0.0   0.0  
                     05         0.002032  1.0  0.0   0.0  

[5 rows x 1046 columns]

In [8]:
y = df.T.reset_index()['species']
y.head()

0    bc
1    bc
2    bc
3    bc
4    bc
Name: species, dtype: object

In [9]:
y.describe()

count     135
unique      6
top        bc
freq       36
Name: species, dtype: object

In [10]:
print(y)

0      bc
1      bc
2      bc
3      bc
4      bc
5      bc
6      bc
7      bc
8      bc
9      bc
10     bc
11     bc
12     ec
13     ec
14     ec
15     ec
16     ec
17     ec
18     ec
19     ec
20     lm
21     lm
22     lm
23     lm
24     lm
25     pa
26     pa
27     pa
28     pa
29     pa
       ..
105    ec
106    ec
107    ec
108    ec
109    lm
110    lm
111    lm
112    pa
113    pa
114    pa
115    pa
116    pa
117    pa
118    pa
119    pa
120    sa
121    sa
122    sa
123    sa
124    sa
125    sa
126    sa
127    sa
128    se
129    se
130    se
131    se
132    se
133    se
134    se
Name: species, Length: 135, dtype: object


In [11]:
from sklearn.utils import shuffle

X, y = shuffle(X, y)

print(X[:5])
print(y[:5])

                                       0         1         2         3  \
species growth_phase replicate                                           
bc      lag          06         0.001939  0.001975  0.001996  0.001918   
pa      stat         07         0.001984  0.001962  0.002017  0.001977   
                     02         0.002022  0.002001  0.002001  0.002012   
bc      stat         07         0.001989  0.001946  0.001970  0.002103   
ec      log          02         0.001940  0.001982  0.002007  0.002073   

                                       4         5         6         7  \
species growth_phase replicate                                           
bc      lag          06         0.001977  0.001925  0.001998  0.001902   
pa      stat         07         0.002017  0.001977  0.002014  0.001943   
                     02         0.001930  0.001940  0.001971  0.002012   
bc      stat         07         0.002013  0.002036  0.002027  0.002046   
ec      log          02         0.001

# PCA + SVM

In [12]:
from sklearn.decomposition import PCA

pca = PCA(n_components=0.99, svd_solver='full')
pca.fit(X)

X_pca = pca.transform(X)
print(X_pca.shape)

(135, 3)


In [13]:
from sklearn.svm import SVC
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score, GridSearchCV

param_grid = {
    'kernel': ['linear', 'rbf'],
    'gamma': [10 ** -n for n in range(10)],
    'C': [10 ** n for n in range(-9, 2)]
}

cv = RepeatedStratifiedKFold(n_splits=3, n_repeats=5)
clf = SVC()

grid_search = GridSearchCV(clf, param_grid, cv=cv, verbose=9, n_jobs=4)
grid_search.fit(X_pca, y)

print(grid_search.best_score_)
print(grid_search.best_params_)

Fitting 15 folds for each of 220 candidates, totalling 3300 fits
[CV] C=1e-09, gamma=1, kernel=linear .................................
[CV] C=1e-09, gamma=1, kernel=linear .................................
[CV] C=1e-09, gamma=1, kernel=linear .................................
[CV] C=1e-09, gamma=1, kernel=linear .................................
[CV]  C=1e-09, gamma=1, kernel=linear, score=0.2553191489361702, total=   0.0s
[CV] C=1e-09, gamma=1, kernel=linear .................................
[CV]  C=1e-09, gamma=1, kernel=linear, score=0.26666666666666666, total=   0.0s
[CV] C=1e-09, gamma=1, kernel=linear .................................
[CV]  C=1e-09, gamma=1, kernel=linear, score=0.27906976744186046, total=   0.0s
[CV]  C=1e-09, gamma=1, kernel=linear, score=0.26666666666666666, total=   0.0s
[CV] C=1e-09, gamma=1, kernel=linear .................................
[CV] C=1e-09, gamma=1, kernel=linear .................................
[CV]  C=1e-09, gamma=1, kernel=linear, score=0.2

[CV] C=1e-09, gamma=0.1, kernel=rbf ..................................
[CV] C=1e-09, gamma=0.01, kernel=linear ..............................
[CV] C=1e-09, gamma=1, kernel=rbf ....................................
[CV]  C=1e-09, gamma=0.1, kernel=rbf, score=0.27906976744186046, total=   0.0s
[CV]  C=1e-09, gamma=0.1, kernel=linear, score=0.26666666666666666, total=   0.0s
[CV] C=1e-09, gamma=0.1, kernel=rbf ..................................
[CV] C=1e-09, gamma=0.1, kernel=linear ...............................
[CV]  C=1e-09, gamma=0.01, kernel=linear, score=0.2553191489361702, total=   0.0s
[CV]  C=1e-09, gamma=1, kernel=rbf, score=0.2553191489361702, total=   0.0s
[CV] C=1e-09, gamma=0.01, kernel=linear ..............................
[CV]  C=1e-09, gamma=0.01, kernel=rbf, score=0.26666666666666666, total=   0.0s
[CV]  C=1e-09, gamma=0.1, kernel=linear, score=0.27906976744186046, total=   0.0s
[CV]  C=1e-09, gamma=0.1, kernel=rbf, score=0.2553191489361702, total=   0.0s
[CV] C=1e-09, g

[CV]  C=1e-09, gamma=0.001, kernel=rbf, score=0.26666666666666666, total=   0.0s
[CV] C=1e-09, gamma=0.01, kernel=rbf .................................
[CV]  C=1e-09, gamma=0.01, kernel=rbf, score=0.27906976744186046, total=   0.0s
[CV] C=1e-09, gamma=0.001, kernel=rbf ................................
[CV] C=1e-09, gamma=0.001, kernel=linear .............................
[CV]  C=1e-09, gamma=0.01, kernel=rbf, score=0.27906976744186046, total=   0.0s
[CV]  C=1e-09, gamma=0.001, kernel=rbf, score=0.27906976744186046, total=   0.0s
[CV] C=1e-09, gamma=0.0001, kernel=linear ............................
[CV] C=1e-09, gamma=0.001, kernel=rbf ................................
[CV]  C=1e-09, gamma=0.001, kernel=linear, score=0.2553191489361702, total=   0.0s
[CV]  C=1e-09, gamma=0.001, kernel=linear, score=0.2553191489361702, total=   0.0s
[CV] C=1e-09, gamma=0.001, kernel=linear .............................
[CV] C=1e-09, gamma=0.001, kernel=linear .............................
[CV] C=1e-09, g

[CV]  C=1e-09, gamma=1e-05, kernel=rbf, score=0.27906976744186046, total=   0.0s
[CV] C=1e-09, gamma=1e-06, kernel=rbf ................................
[CV] C=1e-09, gamma=1e-07, kernel=rbf ................................
[CV] C=1e-09, gamma=1e-05, kernel=rbf ................................
[CV]  C=1e-09, gamma=1e-06, kernel=rbf, score=0.27906976744186046, total=   0.0s
[CV]  C=1e-09, gamma=1e-05, kernel=rbf, score=0.2553191489361702, total=   0.0s
[CV]  C=1e-09, gamma=1e-07, kernel=rbf, score=0.26666666666666666, total=   0.0s
[CV] C=1e-09, gamma=1e-06, kernel=rbf ................................
[CV] C=1e-09, gamma=1e-05, kernel=rbf ................................
[CV] C=1e-09, gamma=1e-07, kernel=rbf ................................
[CV]  C=1e-09, gamma=1e-06, kernel=rbf, score=0.2553191489361702, total=   0.0s
[CV]  C=1e-09, gamma=1e-07, kernel=rbf, score=0.27906976744186046, total=   0.0s
[CV] C=1e-09, gamma=1e-06, kernel=rbf ................................
[CV]  C=1e-09, gamm

[Parallel(n_jobs=4)]: Done  36 tasks      | elapsed:    0.2s


...

[CV]  C=10, gamma=0.0001, kernel=rbf, score=0.27906976744186046, total=   0.0s
[CV] C=10, gamma=0.0001, kernel=rbf ..................................
[CV]  C=10, gamma=0.0001, kernel=rbf, score=0.2553191489361702, total=   0.0s
[CV] C=10, gamma=0.0001, kernel=rbf ..................................
[CV]  C=10, gamma=0.0001, kernel=rbf, score=0.26666666666666666, total=   0.0s
[CV] C=10, gamma=0.0001, kernel=rbf ..................................
[CV]  C=10, gamma=0.0001, kernel=rbf, score=0.27906976744186046, total=   0.0s
[CV] C=10, gamma=0.0001, kernel=rbf ..................................
[CV]  C=10, gamma=0.0001, kernel=rbf, score=0.2553191489361702, total=   0.0s
[CV] C=10, gamma=0.0001, kernel=rbf ..................................
[CV]  C=10, gamma=0.0001, kernel=rbf, score=0.26666666666666666, total=   0.0s
[CV] C=10, gamma=0.0001, kernel=rbf ..................................
[CV]  C=10, gamma=0.0001, kernel=rbf, score=0.27906976744186046, total=   0.0s
[CV] C=10, gamma=0.0001

[CV] C=10, gamma=1e-06, kernel=rbf ...................................
[CV] C=10, gamma=1e-05, kernel=rbf ...................................
[CV]  C=10, gamma=1e-06, kernel=rbf, score=0.2553191489361702, total=   0.0s
[CV]  C=10, gamma=1e-05, kernel=rbf, score=0.27906976744186046, total=   0.0s
[CV] C=10, gamma=1e-05, kernel=rbf ...................................
[CV] C=10, gamma=1e-06, kernel=rbf ...................................
[CV]  C=10, gamma=1e-05, kernel=rbf, score=0.2553191489361702, total=   0.0s
[CV]  C=10, gamma=1e-06, kernel=rbf, score=0.26666666666666666, total=   0.0s
[CV] C=10, gamma=1e-06, kernel=rbf ...................................
[CV] C=10, gamma=1e-05, kernel=rbf ...................................
[CV]  C=10, gamma=1e-06, kernel=rbf, score=0.27906976744186046, total=   0.0s
[CV]  C=10, gamma=1e-05, kernel=rbf, score=0.26666666666666666, total=   0.0s
[CV] C=10, gamma=1e-06, kernel=rbf ...................................
[CV] C=10, gamma=1e-05, kernel=rbf ..

[CV]  C=10, gamma=1e-07, kernel=rbf, score=0.26666666666666666, total=   0.0s
[CV]  C=10, gamma=1e-08, kernel=rbf, score=0.27906976744186046, total=   0.0s
[CV] C=10, gamma=1e-08, kernel=rbf ...................................
[CV] C=10, gamma=1e-07, kernel=rbf ...................................
[CV]  C=10, gamma=1e-08, kernel=rbf, score=0.2553191489361702, total=   0.0s
[CV]  C=10, gamma=1e-07, kernel=rbf, score=0.27906976744186046, total=   0.0s
[CV] C=10, gamma=1e-08, kernel=rbf ...................................
[CV] C=10, gamma=1e-07, kernel=rbf ...................................
[CV]  C=10, gamma=1e-07, kernel=rbf, score=0.2553191489361702, total=   0.0s
[CV]  C=10, gamma=1e-08, kernel=rbf, score=0.26666666666666666, total=   0.0s
[CV] C=10, gamma=1e-07, kernel=rbf ...................................
[CV] C=10, gamma=1e-08, kernel=rbf ...................................
[CV]  C=10, gamma=1e-08, kernel=rbf, score=0.27906976744186046, total=   0.0s
[CV]  C=10, gamma=1e-07, kerne

[CV] C=10, gamma=1e-09, kernel=rbf ...................................
[CV] C=10, gamma=1e-09, kernel=rbf ...................................
[CV]  C=10, gamma=1e-09, kernel=rbf, score=0.26666666666666666, total=   0.0s
[CV]  C=10, gamma=1e-09, kernel=rbf, score=0.2553191489361702, total=   0.0s
[CV] C=10, gamma=1e-09, kernel=rbf ...................................
[CV] C=10, gamma=1e-09, kernel=rbf ...................................
[CV]  C=10, gamma=1e-09, kernel=rbf, score=0.27906976744186046, total=   0.0s
[CV]  C=10, gamma=1e-09, kernel=rbf, score=0.26666666666666666, total=   0.0s
[CV] C=10, gamma=1e-09, kernel=rbf ...................................
[CV] C=10, gamma=1e-09, kernel=rbf ...................................
[CV]  C=10, gamma=1e-09, kernel=rbf, score=0.2553191489361702, total=   0.0s
[CV]  C=10, gamma=1e-09, kernel=rbf, score=0.27906976744186046, total=   0.0s
[CV] C=10, gamma=1e-09, kernel=rbf ...................................
[CV]  C=10, gamma=1e-09, kernel=rbf, 

[Parallel(n_jobs=4)]: Done 3300 out of 3300 | elapsed:   13.4s finished


In [14]:
%%time

cv = RepeatedStratifiedKFold(n_splits=3, n_repeats=20)
scores = cross_val_score(grid_search.best_estimator_, X_pca, y, cv=cv)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.54 (+/- 0.13)
CPU times: user 93.8 ms, sys: 15.6 ms, total: 109 ms
Wall time: 94.8 ms


In [15]:
%%time

cv = RepeatedStratifiedKFold(n_splits=3, n_repeats=20)
scores = cross_val_score(grid_search.best_estimator_, X, y, cv=cv)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.54 (+/- 0.13)
CPU times: user 984 ms, sys: 15.6 ms, total: 1 s
Wall time: 1 s


# Random Forest with Decision Stumps

In [16]:
%%time

from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=1024, max_depth=1)
cv = RepeatedStratifiedKFold(n_splits=3, n_repeats=20)
scores = cross_val_score(clf, X_pca, y, cv=cv)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

/home/anthony/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


Accuracy: 0.49 (+/- 0.11)
CPU times: user 35.2 s, sys: 219 ms, total: 35.4 s
Wall time: 35.4 s


In [17]:
%%time

clf = RandomForestClassifier(n_estimators=1024, max_depth=1)
cv = RepeatedStratifiedKFold(n_splits=3, n_repeats=20)
scores = cross_val_score(clf, X, y, cv=cv)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.38 (+/- 0.09)
CPU times: user 41.8 s, sys: 188 ms, total: 42 s
Wall time: 42 s


# AdaBoost

In [18]:
%%time

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier

clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1),
                        n_estimators=1024)
cv = RepeatedStratifiedKFold(n_splits=3, n_repeats=20)
scores = cross_val_score(clf, X_pca, y, cv=cv)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.34 (+/- 0.13)
CPU times: user 44 s, sys: 15.6 ms, total: 44 s
Wall time: 44 s


In [ ]:
%%time

clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1),
                        n_estimators=1024)
cv = RepeatedStratifiedKFold(n_splits=3, n_repeats=20)
scores = cross_val_score(clf, X, y, cv=cv)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))